## Interesting Visulizations
### Return the top-k interesting visulaizations from any database!
#### Submmitters: Ido Tausi, Afek Nahum

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [42]:

# Importing the dataset
dataset = pd.read_csv('StudentsDepression.csv')
dataset.head(1)

,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33,Visakhapatnam,Student,5,0,8.97,2,0,5-6 hours,Healthy,B.Pharm,Yes,3,1.0,No,1


In [58]:
def bin_numerical_columns(dataframe):
    dataframe = dataframe.copy()
    for column in dataframe.columns:
        if dataframe[column].dtype == 'float64' or dataframe[column].dtype == 'int64':
            if dataframe[column].nunique() < 10:
                continue
            # remove outliers
            q1 = dataframe[column].quantile(0.25)
            q3 = dataframe[column].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            dataframe = dataframe[(dataframe[column] > lower_bound) & (dataframe[column] < upper_bound)]
            # binning
            bins_number = 10
            dataframe[column] = pd.cut(dataframe[column], bins=bins_number)
    return dataframe


In [63]:
def subset_data(dataframe, column, value):
    return dataframe[dataframe[column] == value]

def create_subsets_for_column(dataframe, column):
    subsets = {}
    for value in dataframe[column].unique():
        subsets[value] = subset_data(dataframe, column, value)
    return subsets

def create_subsets(dataframe):
    subsets = {}
    for column in dataframe.columns:
        subsets[column] = create_subsets_for_column(dataframe, column)
    return subsets

binned_dataset = bin_numerical_columns(dataset)
subsets_age = create_subsets_for_column(binned_dataset, 'CGPA')
for key, value in subsets_age.items():
    print(f'{key}: {len(value)}')
    print(type(value))


"""
subsets = {
    "Age": {
        (18, 20]: dataframe,
        (20, 22]: dataframe,
        ...
    },
    "CGPA": {
        (6.5, 7.0]: dataframe,
        (7.0, 7.5]: dataframe,
        ...
    },
    ...

}
"""
    

(8.509, 9.006]: 3422
<class 'pandas.core.frame.DataFrame'>
(5.527, 6.024]: 3771
<class 'pandas.core.frame.DataFrame'>
(7.018, 7.515]: 3139
<class 'pandas.core.frame.DataFrame'>
(8.012, 8.509]: 2903
<class 'pandas.core.frame.DataFrame'>
(9.503, 10.0]: 3614
<class 'pandas.core.frame.DataFrame'>
(6.024, 6.521]: 2261
<class 'pandas.core.frame.DataFrame'>
(7.515, 8.012]: 2458
<class 'pandas.core.frame.DataFrame'>
(6.521, 7.018]: 1797
<class 'pandas.core.frame.DataFrame'>
(5.025, 5.527]: 1898
<class 'pandas.core.frame.DataFrame'>
(9.006, 9.503]: 2617
<class 'pandas.core.frame.DataFrame'>
